In [105]:
from tensorflow import keras
import pandas as pd
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow.keras.backend as kb
import tensorflow as tf
from sklearn import preprocessing

In [252]:


# data
rawDataFileName = 'AnalyzedData/nnDataSource.csv'
pitchControlFileName = 'AnalyzedData/spaceValues.csv'
rawData = pd.read_csv(rawDataFileName, header=0, index_col=0)
pitchControlData = pd.read_csv(pitchControlFileName, header=0, index_col=0)
# display(rawData)

rawCols = rawData.columns
cols = rawCols[4:]
xCols = cols[:-1]
yCols = ["18"]
usableData = rawData[cols]

relData = pd.DataFrame()

xr = usableData[xCols[0]].to_numpy()
yr = usableData[xCols[1]].to_numpy()
vr = usableData[xCols[2]].to_numpy()
ar = usableData[xCols[3]].to_numpy()
orir = usableData[xCols[5]].to_numpy()
dirr = usableData[xCols[6]].to_numpy()
xt = usableData[xCols[7]].to_numpy()
yt = usableData[xCols[8]].to_numpy()
vt = usableData[xCols[9]].to_numpy()
at = usableData[xCols[10]].to_numpy()
orit = usableData[xCols[12]].to_numpy()
dirt = usableData[xCols[13]].to_numpy()
suc = usableData[yCols[0]].to_numpy()

orir = -np.gradient(orir - 90)
dirr = -np.gradient(dirr - 90)
orit = -np.gradient(orit - 90)
dirt = -np.gradient(dirt - 90)

relData['x0'] = xr
relData['y0'] = yr
relData['x1'] = xt - xr
relData['y1'] = yt - yr
relData['vx0'] = vr * np.cos(dirr)
relData['vy0'] = vr * np.sin(dirr)
relData['vx1'] = vt * np.cos(dirt)
relData['vy1'] = vt * np.sin(dirt)
relData['a0'] = ar
relData['a1'] = at
relData['dir0'] = dirr
relData['dir1'] = dirt
relData['ori0'] = orir
relData['ori1'] = orit
relData['pc'] = pitchControlData['pitch control']

relNData = preprocessing.normalize(relData)
# print(relNData)
# normalize by column
# dataRange = [(-10, 130, 140), (0, 53.33, 53.33), (0, 14, 14), (0, 7, 7), (0, 1.4, 1.4), (0, 360, 360), (0, 360, 360)]
# for j in range(2):
#     for i in range(7):
#         col = cols[i + j * 7]
#         usableData[col] = (usableData[col] - dataRange[i][0]) / dataRange[i][2]
# display(usableData)        


# Randomly selecting training data

# indexes
Ntotal = relNData.shape[0]
trainingIndex = np.random.choice(Ntotal, int(Ntotal * 0.90), replace=False)
validationIndex = np.setdiff1d(np.array(range(Ntotal)), trainingIndex)
# data seperation
tXSet = relNData[trainingIndex]
vXSet = relNData[validationIndex]
tYSet = suc[trainingIndex]
vYSet = suc[validationIndex]


# tXSet = relNData[validationIndex]
# vXSet = relNData[validationIndex]
# tYSet = relNData[validationIndex]
# vYSet = relNData[validationIndex]

# display(tSet)
# display(vSet)


In [253]:
# model
from keras import backend as K
initializer = keras.initializers.RandomUniform(minval= -0.2, maxval=0.5)

layers = [keras.layers.Dense(8, activation='elu', input_shape=[15],  kernel_initializer=initializer),
          keras.layers.Dense(16, activation='elu', kernel_initializer=initializer),
          keras.layers.Dense(8, activation='elu', kernel_initializer=initializer),

          
          
          keras.layers.Dense(1, kernel_initializer=initializer)
         ]
model = keras.Sequential(layers)

In [259]:
# compile model
rms = keras.optimizers.RMSprop(0.001)
# model.compile(loss='mean_absolute_error',
#               optimizer='adam', 
#               metrics=['accuracy'])
model.compile(loss='mean_absolute_error',optimizer=rms)

In [260]:
# training
es = keras.callbacks.EarlyStopping(
    monitor='loss', 
    mode='min', 
    patience=80, 
    restore_best_weights=True, 
    verbose=1)
# Add a checkpoint where loss is minimum, and save that model
mc = keras.callbacks.ModelCheckpoint('best_model.SB', monitor='loss', 
                     mode='min',  verbose=0, save_best_only=True)

historyData = model.fit(tXSet, 
                        tYSet,
                        epochs=200,
                        batch_size = 50,
                        callbacks=[es], 
                        verbose=0)

loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1

# print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))
minLoss = np.min(loss_hist)


smallest loss = 0.2313021719455719


In [261]:
# analysis
model.evaluate(tXSet, tYSet)
model.evaluate(vXSet, vYSet)

23/23 [==============================] - 0s 417us/step - loss: 0.2072


0.207227885723114